In [ ]:
!nvidia-smi


In [ ]:
# feature extractoring and preprocessing data
import librosa
import librosa.display
import pandas as pd
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
from pathlib import Path
from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import csv
# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
#Reports
from sklearn.metrics import classification_report, confusion_matrix

import warnings
warnings.filterwarnings('ignore')


## Reading some audio samples

*Applying Pre-Processing Techniques from Example File*

In [ ]:
sr = 16000
br_cl1 = '../input/birdsong-recognition/example_test_audio/BLKFR-10-CPL_20190611_093000.pt540.mp3'
br_cl2 = '../input/birdsong-recognition/example_test_audio/ORANGE-7-CAP_20190606_093000.pt623.mp3'

# 10 seconds of each file
y1,sr = librosa.load(br_cl1, mono=True, sr=sr, offset=0, duration=10)
y2,sr = librosa.load(br_cl2, mono=True, sr=sr, offset=0, duration=10)

In [ ]:
from IPython.display import Audio, IFrame, display

display(Audio(y1,rate=sr))
display(Audio(y2,rate=sr))

As you can hear, the first audio presents a high level background noise, and birds seems far from the mic. In the second audio, bird sounds are much more distinguished from the other noises. We can say that the second audio presents a better SNR (signal-noise ratio).

In [ ]:
librosa.display.waveplot(y1,sr=sr, x_axis='time')



In [ ]:
librosa.display.waveplot(y2,sr=sr, x_axis='time');

If you look to both signal waves, you can see that the first sound presents a lower difference between the background level and some sound events, while this difference is much higher in the second sound. If you do it in sync with the audio, we can see that some bird calls does not appear from the average sound level.

## Logmel-spectogram

A very common preprocessing technique in audio detection applications is to transform audios to its log mel-spectogram representation.
Some concepts here: https://en.wikipedia.org/wiki/Mel-frequency_cepstrum

In [ ]:
S1 = librosa.feature.melspectrogram(y=y1, sr=sr, n_mels=64)
D1 = librosa.power_to_db(S1, ref=np.max)
librosa.display.specshow(D1, x_axis='time', y_axis='mel');

In [ ]:
S2 = librosa.feature.melspectrogram(y=y2, sr=sr, n_mels=64)
D2 = librosa.power_to_db(S2, ref=np.max)
librosa.display.specshow(D2, x_axis='time', y_axis='mel');

The differences become very clear in the log mel spectogram. In the fist case, you can see a lot of artefacts on low frequencies (not birds), and the birds are in levels below the background noises. Besides, background noises are higher in frequencies below 2 kHz.

## Filtering low-frequencies

In [ ]:
from scipy import signal
import random


def f_high(y,sr):
    b,a = signal.butter(10, 2000/(sr/2), btype='highpass')
    yf = signal.lfilter(b,a,y)
    return yf


In [ ]:
yf1 = f_high(y1, sr)
yf2 = f_high(y2, sr)

In [ ]:
librosa.display.waveplot(y1,sr=sr, x_axis='time');
librosa.display.waveplot(yf1,sr=sr, x_axis='time');

In [ ]:
librosa.display.waveplot(y2,sr=sr, x_axis='time');
librosa.display.waveplot(yf2,sr=sr, x_axis='time');

In [ ]:
Sf1 = librosa.feature.melspectrogram(y=yf1, sr=sr, n_mels=64)
Df1 = librosa.power_to_db(Sf1, ref=np.max)
librosa.display.specshow(Df1, x_axis='time', y_axis='mel');

In [ ]:
Sf2 = librosa.feature.melspectrogram(y=yf2, sr=sr, n_mels=64)
Df2 = librosa.power_to_db(Sf2, ref=np.max)
librosa.display.specshow(Df2, x_axis='time', y_axis='mel');

In [ ]:
display(Audio(yf1,rate=sr))
display(Audio(yf2,rate=sr))

## PCEN

PCEN has become a very useful strategy for acoustic event detection, and it has shown to perform better in such tasks as a frontend. Its idea is to perform non-linear compression on time-frequency channels.

I am using the example shown here: https://librosa.org/doc/latest/generated/librosa.pcen.html?highlight=pcen#librosa.pcen

In [ ]:
Dp1 = librosa.pcen(S1 * (2**31), sr=sr, gain=1.1, hop_length=512, bias=2, power=0.5, time_constant=0.8, eps=1e-06, max_size=2)
Dp2 = librosa.pcen(S2 * (2**31), sr=sr, gain=1.1, hop_length=512, bias=2, power=0.5, time_constant=0.8, eps=1e-06, max_size=2)

In [ ]:
librosa.display.specshow(Dp1, x_axis='time', y_axis='mel');

In [ ]:
librosa.display.specshow(Dp2, x_axis='time', y_axis='mel');

In [ ]:
yp1 = librosa.feature.inverse.mel_to_audio(Dp1)
yp2 = librosa.feature.inverse.mel_to_audio(Dp2)

In [ ]:
librosa.display.waveplot(yp1,sr=sr, x_axis='time');
librosa.display.waveplot(y1,sr=sr, x_axis='time');

In [ ]:
librosa.display.waveplot(yp2,sr=sr, x_axis='time');
librosa.display.waveplot(y2,sr=sr, x_axis='time');

In [ ]:
display(Audio(yp1,rate=sr))
display(Audio(yp2,rate=sr))

## Spectral Gating

In [ ]:
import time
from datetime import timedelta as td


def _stft(y, n_fft, hop_length, win_length):
    return librosa.stft(y=y, n_fft=n_fft, hop_length=hop_length, win_length=win_length)


def _istft(y, hop_length, win_length):
    return librosa.istft(y, hop_length, win_length)


def _amp_to_db(x):
    return librosa.core.amplitude_to_db(x, ref=1.0, amin=1e-20, top_db=80.0)


def _db_to_amp(x,):
    return librosa.core.db_to_amplitude(x, ref=1.0)


def plot_spectrogram(signal, title):
    fig, ax = plt.subplots(figsize=(20, 4))
    cax = ax.matshow(
        signal,
        origin="lower",
        aspect="auto",
        cmap=plt.cm.seismic,
        vmin=-1 * np.max(np.abs(signal)),
        vmax=np.max(np.abs(signal)),
    )
    fig.colorbar(cax)
    ax.set_title(title)
    plt.tight_layout()
    plt.show()


def plot_statistics_and_filter(
    mean_freq_noise, std_freq_noise, noise_thresh, smoothing_filter
):
    fig, ax = plt.subplots(ncols=2, figsize=(20, 4))
    plt_mean, = ax[0].plot(mean_freq_noise, label="Mean power of noise")
    plt_std, = ax[0].plot(std_freq_noise, label="Std. power of noise")
    plt_std, = ax[0].plot(noise_thresh, label="Noise threshold (by frequency)")
    ax[0].set_title("Threshold for mask")
    ax[0].legend()
    cax = ax[1].matshow(smoothing_filter, origin="lower")
    fig.colorbar(cax)
    ax[1].set_title("Filter for smoothing Mask")
    plt.show()

def removeNoise(
    audio_clip,
    noise_clip,
    n_grad_freq=2,
    n_grad_time=4,
    n_fft=2048,
    win_length=2048,
    hop_length=512,
    n_std_thresh=1.5,
    prop_decrease=1.0,
    verbose=False,
    visual=False,
):
    """Remove noise from audio based upon a clip containing only noise

    Args:
        audio_clip (array): The first parameter.
        noise_clip (array): The second parameter.
        n_grad_freq (int): how many frequency channels to smooth over with the mask.
        n_grad_time (int): how many time channels to smooth over with the mask.
        n_fft (int): number audio of frames between STFT columns.
        win_length (int): Each frame of audio is windowed by `window()`. The window will be of length `win_length` and then padded with zeros to match `n_fft`..
        hop_length (int):number audio of frames between STFT columns.
        n_std_thresh (int): how many standard deviations louder than the mean dB of the noise (at each frequency level) to be considered signal
        prop_decrease (float): To what extent should you decrease noise (1 = all, 0 = none)
        visual (bool): Whether to plot the steps of the algorithm

    Returns:
        array: The recovered signal with noise subtracted

    """
    if verbose:
        start = time.time()
    # STFT over noise
    noise_stft = _stft(noise_clip, n_fft, hop_length, win_length)
    noise_stft_db = _amp_to_db(np.abs(noise_stft))  # convert to dB
    # Calculate statistics over noise
    mean_freq_noise = np.mean(noise_stft_db, axis=1)
    std_freq_noise = np.std(noise_stft_db, axis=1)
    noise_thresh = mean_freq_noise + std_freq_noise * n_std_thresh
    if verbose:
        print("STFT on noise:", td(seconds=time.time() - start))
        start = time.time()
    # STFT over signal
    if verbose:
        start = time.time()
    sig_stft = _stft(audio_clip, n_fft, hop_length, win_length)
    sig_stft_db = _amp_to_db(np.abs(sig_stft))
    if verbose:
        print("STFT on signal:", td(seconds=time.time() - start))
        start = time.time()
    # Calculate value to mask dB to
    mask_gain_dB = np.min(_amp_to_db(np.abs(sig_stft)))
    #print(noise_thresh, mask_gain_dB)
    # Create a smoothing filter for the mask in time and frequency
    smoothing_filter = np.outer(
        np.concatenate(
            [
                np.linspace(0, 1, n_grad_freq + 1, endpoint=False),
                np.linspace(1, 0, n_grad_freq + 2),
            ]
        )[1:-1],
        np.concatenate(
            [
                np.linspace(0, 1, n_grad_time + 1, endpoint=False),
                np.linspace(1, 0, n_grad_time + 2),
            ]
        )[1:-1],
    )
    smoothing_filter = smoothing_filter / np.sum(smoothing_filter)
    # calculate the threshold for each frequency/time bin
    db_thresh = np.repeat(
        np.reshape(noise_thresh, [1, len(mean_freq_noise)]),
        np.shape(sig_stft_db)[1],
        axis=0,
    ).T
    # mask if the signal is above the threshold
    sig_mask = sig_stft_db < db_thresh
    if verbose:
        print("Masking:", td(seconds=time.time() - start))
        start = time.time()
    # convolve the mask with a smoothing filter
    sig_mask = scipy.signal.fftconvolve(sig_mask, smoothing_filter, mode="same")
    sig_mask = sig_mask * prop_decrease
    if verbose:
        print("Mask convolution:", td(seconds=time.time() - start))
        start = time.time()
    # mask the signal
    sig_stft_db_masked = (
        sig_stft_db * (1 - sig_mask)
        + np.ones(np.shape(mask_gain_dB)) * mask_gain_dB * sig_mask
    )  # mask real
    sig_imag_masked = np.imag(sig_stft) * (1 - sig_mask)
    sig_stft_amp = (_db_to_amp(sig_stft_db_masked) * np.sign(sig_stft)) + (
        1j * sig_imag_masked
    )
    if verbose:
        print("Mask application:", td(seconds=time.time() - start))
        start = time.time()
    # recover the signal
    recovered_signal = _istft(sig_stft_amp, hop_length, win_length)
    recovered_spec = _amp_to_db(
        np.abs(_stft(recovered_signal, n_fft, hop_length, win_length))
    )
    if verbose:
        print("Signal recovery:", td(seconds=time.time() - start))
    if visual:
        plot_spectrogram(noise_stft_db, title="Noise")
    if visual:
        plot_statistics_and_filter(
            mean_freq_noise, std_freq_noise, noise_thresh, smoothing_filter
        )
    if visual:
        plot_spectrogram(sig_stft_db, title="Signal")
    if visual:
        plot_spectrogram(sig_mask, title="Mask applied")
    if visual:
        plot_spectrogram(sig_stft_db_masked, title="Masked signal")
    if visual:
        plot_spectrogram(recovered_spec, title="Recovered spectrogram")
    return recovered_signal

In [ ]:
noise1 = y1[5*sr:6*sr]
yg1 = removeNoise(audio_clip=y1, noise_clip=noise1,
    n_grad_freq=2,
    n_grad_time=4,
    n_fft=2048,
    win_length=2048,
    hop_length=512,
    n_std_thresh=1.5,
    prop_decrease=1.0,
    verbose=False,
    visual=False)
noise2 = y2[0:1*sr]
yg2 = removeNoise(audio_clip=y2, noise_clip=noise2,
    n_grad_freq=2,
    n_grad_time=4,
    n_fft=2048,
    win_length=2048,
    hop_length=512,
    n_std_thresh=2.5,
    prop_decrease=1.0,
    verbose=False,
    visual=False)

In [ ]:
librosa.display.waveplot(y1,sr=sr, x_axis='time');
librosa.display.waveplot(yg1,sr=sr, x_axis='time');

In [ ]:
librosa.display.waveplot(y2,sr=sr, x_axis='time');
librosa.display.waveplot(yg2,sr=sr, x_axis='time');

In [ ]:
Sg1 = librosa.feature.melspectrogram(y=yg1, sr=sr, n_mels=64)
Dg1 = librosa.power_to_db(Sg1, ref=np.max)
librosa.display.specshow(Dg1, x_axis='time', y_axis='mel');

In [ ]:
Sg2 = librosa.feature.melspectrogram(y=yg2, sr=sr, n_mels=64)
Dg2 = librosa.power_to_db(Sg2, ref=np.max)
librosa.display.specshow(Dg2, x_axis='time', y_axis='mel');

In [ ]:
display(Audio(yg1,rate=sr))
display(Audio(yg2,rate=sr))

In [ ]:
# loading train data
train_df = pd.read_csv('../input/birdsong-recognition/train.csv')
test_df = pd.read_csv('../input/birdsong-recognition/test.csv')
train_df.head(3)

In [ ]:
# checking the train data set
train_df.info()

In [ ]:
import seaborn as sns
sns.set_style('darkgrid')
fig = plt.figure(figsize=(15,5))
ax1 = fig.add_subplot(121)
sns.distplot(train_df.species.value_counts(),ax=ax1)
ax1.set_title("Distribution of number of bird species")
ax1.set_xlabel("Count of bird species")

counts, bin_edges = np.histogram(train_df.species.value_counts(), bins=10, density = True)

# plotting cdf 
# https://stackoverflow.com/questions/10640759/how-to-get-the-cumulative-distribution-function-with-numpy
pdf = counts/(sum(counts))
cdf = np.cumsum(pdf)
ax2 = fig.add_subplot(122)
ax2.plot(bin_edges[1:], cdf)
ax2.set_title("Commulative distribution of count of the species")
ax2.set_xlabel("Count")
ax2.set_ylabel("percentage")
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
ax = sns.countplot(x='rating',data=train_df)
plt.title("Distribution of Ratings of the Audio", fontsize=16)
plt.xlabel("ratings")
plt.show()

****Predictive Modelling****

Importing Libraries

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import zipfile
from IPython.display import Audio
from pydub import AudioSegment
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import pickle
import random
from tqdm import tqdm
import datetime
#from audiomentations import *
import multiprocessing
import cv2
# # library used to process the audio files
import librosa
import librosa.display
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input, Dense, Conv2D, Softmax, LSTM, Embedding, concatenate
from tensorflow.keras.layers import Dropout, BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50, InceptionV3, VGG16
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau, EarlyStopping
from IPython.display import Image

Data Loading

In [ ]:
# loading train data
train_df = pd.read_csv('../input/birdsong-recognition/train.csv')
test_df = pd.read_csv('../input/birdsong-recognition/example_test_audio_summary.csv')
train_df.head()

In [ ]:
train_df.shape

In [ ]:
null_columns=train_df.columns[train_df.isnull().any()]
train_df[null_columns].isnull().sum()

In [ ]:
data4=list()

In [ ]:
import os

train_audio_path = '../input/birdsong-recognition/train_audio'
print("train_audio exists: ", os.path.exists(train_audio_path))

for dirname, _, filenames in os.walk(train_audio_path):
    _.sort()
    for filename in sorted(filenames):
        print(os.path.join(dirname, filename))
        data4.append((dirname,filename))
        
df2= pd.DataFrame(data4, columns=['dirnames','filename'])



In [ ]:
df2.shape

In [ ]:
df2.head()

In [ ]:
df2['directory'] = df2['dirnames'].str.cat(df2['filename'], sep ="/") 
print(df2)

In [ ]:
df2.tail()

In [ ]:
df2.isnull().sum

In [ ]:
# concatenating the DataFrames 
df3 = pd.concat([train_df, df2], join = 'outer', axis = 1) 
  


In [ ]:
df3.isnull().sum

In [ ]:
df3.tail()

In [ ]:
train_df=df3

In [ ]:
train_df.head()

In [ ]:
test_df.head()

Analyzing the Data

In [ ]:
# checking the train data set
train_df.info()

In [ ]:
# Checking the unique types of bird species in the data
print("Number of bird species:",len(train_df['species'].unique()))

In [ ]:
y_labels = train_df['ebird_code'].unique()
print("Bird species in abbrebviated: ",y_labels[:5])
print("BIrd Speccies: ",train_df['species'].unique()[:5])

Note: The birdnames have been abbreviated, and there are 263 unique classes.

Encoding & Preparing the Dataset

In [ ]:
# dataframe containg the path to the audio files and the class labels
df_audio = train_df[['filename','directory','duration','sampling_rate','ebird_code']]
df_audio.to_csv('df_audio.csv',index=False)


In [ ]:

df_audio = pd.read_csv('df_audio.csv')
df_audio.head(2)

In [ ]:
df_audio = pd.read_csv('df_audio.csv')
df_audio.head()

In [ ]:

#split the data into train and validation and save in X_train, X_cv, y_train, y_cv
#use stratify sampling,random state of 45 and validaton size of 20%
X_train, X_test, y_train, y_test = train_test_split(df_audio,df_audio.ebird_code,test_size=0.2, 
                                                    random_state=45,stratify=df_audio.ebird_code)

print("Training data: ",X_train.shape,y_train.shape)
print("Validataion data: ",X_test.shape,y_test.shape)



In [ ]:
X_train.head()

In [ ]:

bird_species_mapping = dict(zip(train_df['ebird_code'],train_df['species']))
# Integer coding the class labels
ebird_code_labeled = dict()
ebird_code_classes = dict()

ebird_code_labeled['nocall'] = 0
ebird_code_classes[0] = 'nocall'
for i, bird in enumerate(df_audio['ebird_code'].unique()):
    ebird_code_labeled[bird] = i+1
    ebird_code_classes[i+1] = bird
    
# pickle.dump((bird_species_mapping,ebird_code_labeled,ebird_code_classes), open('model_logs/bird_species_code.pkl','wb'))

Generating samples of 5 seconds from the audio files. (Many audio files don't have multiple of 20 second split)

In [ ]:
# sampling audio to 5 seconds length
def raw_data(filename,bird_code,raw_df):
    sample_rate = 44100
    try:
        samples, sample_rate = librosa.load(filename,sr=44100)
    except:
        sound = AudioSegment.from_file(filename)
        samples = sound.get_array_of_samples()
        new_sound = sound._spawn(samples)
        samples = np.array(samples).astype(np.float32)

    samples, _ = librosa.effects.trim(samples)
    data_point_per_second = 10
    
    #Take 10 data points every second
    samples = samples[0::int(sample_rate/data_point_per_second)]
    #We normalize each sample before extracting 5s samples from it
    normalized_sample = sklearn.preprocessing.minmax_scale(samples, axis=0)
    
    #only take 5s samples and add them to the dataframe
    song_sample = []
    sample_length = 5*data_point_per_second
    for idx in range(0,len(normalized_sample),sample_length): 
        song_sample = normalized_sample[idx:idx+sample_length]
        if len(song_sample)>=sample_length:
            raw_df = raw_df.append({"song_samples":np.asarray(song_sample).astype(np.float32),
                                            "bird_code":ebird_code_labeled[bird_code]}, ignore_index=True)
    return raw_df

In [ ]:
'''
raw_df = pd.DataFrame(columns=["song_samples","bird_code"])
for i, row in tqdm(df_audio[['directory','ebird_code']][12327:].iterrows()):
    raw_df = raw_data(row.directory, row.ebird_code, raw_df)
    
raw_df.to_csv('raw_df.csv',index=False)

In [ ]:
# stacking the image to 3 channels for CNN models
def mono_to_color(X: np.ndarray, mean=None, std=None, norm_max=None, norm_min=None, eps=1e-6):
    # Stack X as [X,X,X]
    X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    X = X - mean
    std = std or X.std()
    Xstd = X / (std + eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > eps:
        # Normalize to [0, 255]
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

In [ ]:

os.makedirs('./processed_data/ResNet/tr/images_data')
os.makedirs('./processed_data/Inception/tr/images_data')
os.makedirs('./processed_data/spec_data/tr/images_data')

#test dir
os.makedirs('./processed_data/ResNet/val/images_data')
os.makedirs('./processed_data/Inception/val/images_data')
os.makedirs('./processed_data/spec_data/val/images_data')


In [ ]:
!ls ..

In [ ]:
# sampling audio to 5 seconds length
def spectrogram_images(directory,bird_code):
    sample_rate = 44100
    try:
        samples, sample_rate = librosa.load(directory,sr=44100)
    except:
        sound = AudioSegment.from_file(directory)
        samples = sound.get_array_of_samples()
        new_sound = sound._spawn(samples)
        samples = np.array(samples).astype(np.float32)

    samples, _ = librosa.effects.trim(samples)
    
    #only take 5s samples and save them to the np array
    sample_length = 5*sample_rate
    X = np.abs(librosa.stft(samples[:44100*5], n_fft=1024, hop_length=512))
    spectrogram = librosa.amplitude_to_db(X)
    image = mono_to_color(spectrogram)

    res = cv2.resize(image, (224, 224))
    res = (res / 255.0).astype(np.float32)
    res = np.asarray(res)
    
    incep = cv2.resize(image, (299, 299))
    incep = (incep / 255.0).astype(np.float32)
    incep = np.asarray(incep)
    
    label = ebird_code_labeled[bird_code]
    
    return spectrogram,res,incep,label


def func(start,end,idx):
    labels = []
    img_res = []
    img_incep = []
    spec = []

    for i, row in X_train[['directory','ebird_code']][start*idx:end*(idx+1)].iterrows():
        mel_spec,res,incep,label = spectrogram_images(row.directory, row.ebird_code)
        spec.extend(mel_spec)
        img_res.append(res)
        img_incep.append(incep)
        labels.append(label)
    
    # stroing the data as npz file for easy access
#     np.savez_compressed('processed_data/VGG/STFT/train/images_data'+str(idx),a=spec,b=labels)
    np.savez_compressed('processed_data/ResNet/tr/images_data'+str(idx),a=img_res,b=labels)
    np.savez_compressed('processed_data/Inception/tr/images_data'+str(idx),a=img_incep,b=labels)
    np.savez_compressed('processed_data/spec_data/tr/images_data'+str(idx),a=spec,b=labels)
    
def func1(start,end,idx):
    labels = []
    img_res = []
    img_incep = []
    spec = []

    for i, row in X_cv[['directory','ebird_code']][start*idx:end*(idx+1)].iterrows():
        mel_spec,res,incep,label = spectrogram_images(row.directory, row.ebird_code)
        spec.extend(mel_spec)
        img_res.append(res)
        img_incep.append(incep)
        labels.append(label)
        
#     np.savez_compressed('processed_data/VGG/STFT/validation/images_data'+str(idx),a=spec,b=labels)    
    np.savez_compressed('processed_data/ResNet/val/images_data'+str(idx),a=img_res,b=labels)
    np.savez_compressed('processed_data/Inception/val/images_data'+str(idx),a=img_incep,b=labels)
    np.savez_compressed('processed_data/spec_data/val/images_data'+str(idx),a=spec,b=labels)

In [ ]:

for i in tqdm(range(0,535,8)):
    if i != 528:
        p1 = multiprocessing.Process(target=func, args=(32,32,i))
        p2 = multiprocessing.Process(target=func, args=(32,32,i+1))
        p3 = multiprocessing.Process(target=func, args=(32,32,i+2))
        p4 = multiprocessing.Process(target=func, args=(32,32,i+3))
        p5 = multiprocessing.Process(target=func, args=(32,32,i+4))
        p6 = multiprocessing.Process(target=func, args=(32,32,i+5))
        p7 = multiprocessing.Process(target=func, args=(32,32,i+6))
        p8 = multiprocessing.Process(target=func, args=(32,32,i+7))

        p1.start()
        p2.start()
        p3.start()
        p4.start()
        p5.start()
        p6.start()
        p7.start()
        p8.start()

        p1.join()
        p2.join()
        p3.join()
        p4.join()
        p5.join()
        p6.join()
        p7.join()
        p8.join()
    else:
        p1 = multiprocessing.Process(target=func, args=(32,32,i))
        p2 = multiprocessing.Process(target=func, args=(32,32,i+1))
        p3 = multiprocessing.Process(target=func, args=(32,32,i+2))
        p4 = multiprocessing.Process(target=func, args=(32,32,i+3))
        p5 = multiprocessing.Process(target=func, args=(32,32,i+4))
        p6 = multiprocessing.Process(target=func, args=(32,32,i+5))
        p7 = multiprocessing.Process(target=func, args=(32,32,i+6))

        p1.start()
        p2.start()
        p3.start()
        p4.start()
        p5.start()
        p6.start()
        p7.start()

        p1.join()
        p2.join()
        p3.join()
        p4.join()
        p5.join()
        p6.join()
        p7.join()

In [ ]:
print('ac')

In [ ]:

# train set
train_dir_path = "./processed_data/Inception/tr"
train_files_path = os.listdir(train_dir_path)
train_gen_res = DataGenerator(train_dir_path, train_files_path)

# test set
test_dir_path = "./processed_data/Inception/val"
test_files_path = os.listdir(test_dir_path)
test_gen_res = DataGenerator(train_dir_path, test_files_path)

In [ ]:
os.makedirs('/kaggle/tmp')


In [ ]:
from distutils.dir_util import copy_tree

fromDir='./processed_data'
toDir='/kaggle/tmp'

copy_tree(fromDir,toDir)

In [ ]:
import shutil
shutil.make_archive('/kaggle/tmp', 'zip', '/kaggle/tmp')

In [ ]:
from IPython.display import FileLink
FileLink('/kaggle/tmp.zip')

In [ ]:
!ls